In [ ]:
pip install pyspark

     |████████████████████████████████| 212.3MB 70kB/s 
     |████████████████████████████████| 204kB 41.2MB/s 
  Created wheel for pyspark: filename=pyspark-3.1.1-py2.py3-none-any.whl size=212767604 sha256=0074fdbbef5350fa812e2fcc51a00ccef32673f75735b0985cdd50545a9cb488
  Stored in directory: /root/.cache/pip/wheels/0b/90/c0/01de724414ef122bd05f056541fb6a0ecf47c7ca655f8b3c0f
Successfully built pyspark


In [ ]:
from pyspark import SparkConf, SparkContext
import collections
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName('Machine Learning').getOrCreate()

In [ ]:
df = spark.read.csv('bank.csv',header=True, inferSchema= True)
df.printSchema()

root
 |-- age: integer (nullable = true)
 |-- job: string (nullable = true)
 |-- marital: string (nullable = true)
 |-- education: string (nullable = true)
 |-- default: string (nullable = true)
 |-- balance: integer (nullable = true)
 |-- housing: string (nullable = true)
 |-- loan: string (nullable = true)
 |-- contact: string (nullable = true)
 |-- day: integer (nullable = true)
 |-- month: string (nullable = true)
 |-- duration: integer (nullable = true)
 |-- campaign: integer (nullable = true)
 |-- pdays: integer (nullable = true)
 |-- previous: integer (nullable = true)
 |-- poutcome: string (nullable = true)
 |-- deposit: string (nullable = true)



In [ ]:
df.show()

+---+-----------+--------+---------+-------+-------+-------+----+-------+---+-----+--------+--------+-----+--------+--------+-------+
|age|        job| marital|education|default|balance|housing|loan|contact|day|month|duration|campaign|pdays|previous|poutcome|deposit|
+---+-----------+--------+---------+-------+-------+-------+----+-------+---+-----+--------+--------+-----+--------+--------+-------+
| 59|     admin.| married|secondary|     no|   2343|    yes|  no|unknown|  5|  may|    1042|       1|   -1|       0| unknown|    yes|
| 56|     admin.| married|secondary|     no|     45|     no|  no|unknown|  5|  may|    1467|       1|   -1|       0| unknown|    yes|
| 41| technician| married|secondary|     no|   1270|    yes|  no|unknown|  5|  may|    1389|       1|   -1|       0| unknown|    yes|
| 55|   services| married|secondary|     no|   2476|    yes|  no|unknown|  5|  may|     579|       1|   -1|       0| unknown|    yes|
| 54|     admin.| married| tertiary|     no|    184|     no|  

In [ ]:
import pandas as pd
pd.DataFrame(df.take(5),columns = df.columns).transpose()

,0,1,2,3,4
age,59,56,41,55,54
job,admin.,admin.,technician,services,admin.
marital,married,married,married,married,married
education,secondary,secondary,secondary,secondary,tertiary
default,no,no,no,no,no
balance,2343,45,1270,2476,184
housing,yes,no,yes,yes,no
loan,no,no,no,no,no
contact,unknown,unknown,unknown,unknown,unknown
day,5,5,5,5,5


In [ ]:
from pyspark.ml.feature import StringIndexer
indexer = StringIndexer()
indexer.setInputCol('deposit').setOutputCol("label")
df1 = indexer.fit(df).transform(df)

In [ ]:
df1.show()

+---+-----------+--------+---------+-------+-------+-------+----+-------+---+-----+--------+--------+-----+--------+--------+-------+-----+
|age|        job| marital|education|default|balance|housing|loan|contact|day|month|duration|campaign|pdays|previous|poutcome|deposit|label|
+---+-----------+--------+---------+-------+-------+-------+----+-------+---+-----+--------+--------+-----+--------+--------+-------+-----+
| 59|     admin.| married|secondary|     no|   2343|    yes|  no|unknown|  5|  may|    1042|       1|   -1|       0| unknown|    yes|  1.0|
| 56|     admin.| married|secondary|     no|     45|     no|  no|unknown|  5|  may|    1467|       1|   -1|       0| unknown|    yes|  1.0|
| 41| technician| married|secondary|     no|   1270|    yes|  no|unknown|  5|  may|    1389|       1|   -1|       0| unknown|    yes|  1.0|
| 55|   services| married|secondary|     no|   2476|    yes|  no|unknown|  5|  may|     579|       1|   -1|       0| unknown|    yes|  1.0|
| 54|     admin.| ma

In [ ]:
class_name = df.columns[len(df.columns)-1]
df.groupby(class_name).count().toPandas()

,deposit,count
0,no,5873
1,yes,5289


In [ ]:
from pyspark.ml.feature import StringIndexer
indexer = StringIndexer()
indexer.setInputCol(class_name).setOutputCol('label')
dfl = indexer.fit(df).transform(df)

In [ ]:
dfl.show()

+---+-----------+--------+---------+-------+-------+-------+----+-------+---+-----+--------+--------+-----+--------+--------+-------+-----+
|age|        job| marital|education|default|balance|housing|loan|contact|day|month|duration|campaign|pdays|previous|poutcome|deposit|label|
+---+-----------+--------+---------+-------+-------+-------+----+-------+---+-----+--------+--------+-----+--------+--------+-------+-----+
| 59|     admin.| married|secondary|     no|   2343|    yes|  no|unknown|  5|  may|    1042|       1|   -1|       0| unknown|    yes|  1.0|
| 56|     admin.| married|secondary|     no|     45|     no|  no|unknown|  5|  may|    1467|       1|   -1|       0| unknown|    yes|  1.0|
| 41| technician| married|secondary|     no|   1270|    yes|  no|unknown|  5|  may|    1389|       1|   -1|       0| unknown|    yes|  1.0|
| 55|   services| married|secondary|     no|   2476|    yes|  no|unknown|  5|  may|     579|       1|   -1|       0| unknown|    yes|  1.0|
| 54|     admin.| ma

In [ ]:
(training_data, test_data) = dfl.randomSplit([0.8, 0.2])

In [ ]:
training_data.toPandas()

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,deposit,label
0,18,student,single,primary,no,608,no,no,cellular,12,aug,267,1,-1,0,unknown,yes,1.0
1,18,student,single,primary,no,608,no,no,cellular,13,nov,210,1,93,1,success,yes,1.0
2,18,student,single,secondary,no,5,no,no,cellular,24,aug,143,2,-1,0,unknown,no,0.0
3,18,student,single,unknown,no,108,no,no,cellular,8,sep,169,1,-1,0,unknown,yes,1.0
4,18,student,single,unknown,no,108,no,no,cellular,10,aug,167,1,-1,0,unknown,yes,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8913,90,retired,divorced,secondary,no,1,no,no,cellular,13,feb,152,3,-1,0,unknown,yes,1.0
8914,92,retired,married,unknown,no,775,no,no,cellular,26,jan,164,4,96,3,success,yes,1.0
8915,93,retired,married,unknown,no,775,no,no,cellular,4,aug,476,2,13,9,success,yes,1.0
8916,93,retired,married,unknown,no,775,no,no,cellular,22,jul,860,2,177,7,success,yes,1.0


In [ ]:
class_name = 'label'
feature_names = df.columns[:-1]
print(class_name)
print(feature_names)

label
['age', 'job', 'marital', 'education', 'default', 'balance', 'housing', 'loan', 'contact', 'day', 'month', 'duration', 'campaign', 'pdays', 'previous', 'poutcome']


In [ ]:
from pyspark.ml.feature import VectorAssembler
assembler = VectorAssembler()
assembler.setInputCols(feature_names).setOutputCol('features')
transformed_data = assembler.transform(dfl)

VectorAssembler_c2d495633b8f

In [ ]:
transformed_data.show()

+---+---+---+---+-----------+-----+-----------------+
| f0| f1| f2| f3|         f4|label|         features|
+---+---+---+---+-----------+-----+-----------------+
|4.9|3.0|1.4|0.2|Iris-setosa|  2.0|[4.9,3.0,1.4,0.2]|
|4.7|3.2|1.3|0.2|Iris-setosa|  2.0|[4.7,3.2,1.3,0.2]|
|4.6|3.1|1.5|0.2|Iris-setosa|  2.0|[4.6,3.1,1.5,0.2]|
|5.0|3.6|1.4|0.2|Iris-setosa|  2.0|[5.0,3.6,1.4,0.2]|
|5.4|3.9|1.7|0.4|Iris-setosa|  2.0|[5.4,3.9,1.7,0.4]|
|4.6|3.4|1.4|0.3|Iris-setosa|  2.0|[4.6,3.4,1.4,0.3]|
|5.0|3.4|1.5|0.2|Iris-setosa|  2.0|[5.0,3.4,1.5,0.2]|
|4.4|2.9|1.4|0.2|Iris-setosa|  2.0|[4.4,2.9,1.4,0.2]|
|4.9|3.1|1.5|0.1|Iris-setosa|  2.0|[4.9,3.1,1.5,0.1]|
|5.4|3.7|1.5|0.2|Iris-setosa|  2.0|[5.4,3.7,1.5,0.2]|
|4.8|3.4|1.6|0.2|Iris-setosa|  2.0|[4.8,3.4,1.6,0.2]|
|4.8|3.0|1.4|0.1|Iris-setosa|  2.0|[4.8,3.0,1.4,0.1]|
|4.3|3.0|1.1|0.1|Iris-setosa|  2.0|[4.3,3.0,1.1,0.1]|
|5.8|4.0|1.2|0.2|Iris-setosa|  2.0|[5.8,4.0,1.2,0.2]|
|5.7|4.4|1.5|0.4|Iris-setosa|  2.0|[5.7,4.4,1.5,0.4]|
|5.4|3.9|1.3|0.4|Iris-setosa

In [ ]:
(training_data, test_data) = transformed_data.randomSplit([0.8,0.2])

In [ ]:
from pyspark.ml.classification import LogisticRegression
model = LogisticRegression(featuresCol = 'features',labelCol=class_name, maxIter=30)

In [ ]:
M = model.fit(training_data)

In [ ]:
predictions = M.transform(test_data)

In [ ]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

multi_evaluator = MulticlassClassificationEvaluator(labelCol = 'label', metricName = 'accuracy')
print('Logistic Regression Accuracy:', multi_evaluator.evaluate(predictions))

Logistic Regression Accuracy: 1.0
